In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
import nltk
from nltk import *
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

def pickle_file(filename, obj):
    with open('dumps/' + filename, 'wb') as f:
        pickle.dump(obj, f)


def unpickle_file(filename):
    with open('dumps/' + filename, 'rb') as f:
        return pickle.load(f)
    
def print_classification_report(true, pred):
    print(classification_report(true, pred))
    

In [7]:
def preprocess(text):
# Remove punctuation, stopword and then stemmering
    stop = set(stopwords.words("english"))
    stemmer = stem.PorterStemmer()
    punctuation = set(string.punctuation)

    doc = [stemmer.stem(word) for word in nltk.word_tokenize(text) if (word not in punctuation) and (word not in stop)]

    doc = ' '.join(w for w in doc)
    return doc

# data_train = pd.read_csv('data_train.csv', encoding='utf-8')

# data_train["Summary"] = data_train["Summary"].apply(preprocess)

# summaries_train = data_train['Summary'].as_matrix()

# ydf = data_train.drop('Summary', axis = 1)
# ydf = ydf.drop('Unnamed: 0', axis = 1)
# ydf = ydf.as_matrix()

# tfidfVect = TfidfVectorizer()
# tfidf = tfidfVect.fit_transform(summaries_train)

# pickle_file('tfidf.dat', tfidf)
# pickle_file('ydf.dat', ydf)



In [25]:
tfidf = unpickle_file('tfidf.dat')
ydf = unpickle_file('ydf.dat')
X_train = tfidf
y_train = ydf

In [23]:
def train_linear_svc(features_train, labels_train):
    print ("Training linear SVC")
    clf = LinearSVC(C=1, dual=True, class_weight='balanced')
    #         clf.fit(features_train, labels_train)
    clf = OneVsRestClassifier(clf).fit(features_train, labels_train)
    return clf


def train_randomForest(features_train, labels_train):
    print ("Training random forest")
    clf = RandomForestClassifier(n_estimators=70, max_depth=70)
    clf = OneVsRestClassifier(clf).fit(features_train, labels_train)
    return clf

def train_decisionTree(features_train, labels_train):
    print ("Training decision tree")
    clf = tree.DecisionTreeClassifier()
    clf = OneVsRestClassifier(clf).fit(features_train, labels_train)
    return clf

def train_adaboost_decision_tree(features_train, labels_train):
    print ("Training adaboost decision tree")
    clf = AdaBoostClassifier( tree.DecisionTreeClassifier(max_depth=2),
                              n_estimators=600,
                              learning_rate=1)
    clf = OneVsRestClassifier(clf).fit(features_train, labels_train)
    return clf

In [21]:
data_test = pd.read_csv('data_test.csv', encoding='utf-8')

data_test["Summary"] = data_test["Summary"].apply(preprocess)

summaries_test = data_test['Summary'].as_matrix()

ydf_test = pd.read_csv('labels_test.csv', encoding='utf-8')
ydf_test = ydf_test.drop('Unnamed: 0', axis = 1)
ydf_test = ydf_test.as_matrix()

In [ ]:
mlb = MultiLabelBinarizer()
y_enc = mlb.fit_transform(y_train)

svm_classifier_linear = train_linear_svc(X_train, y_enc)
pickle_file('svm_classifier_linear', svm_classifier_linear)
tfidfVect = TfidfVectorizer()
tfidf = tfidfVect.fit_transform(summaries_test)
labels_pred = svm_classifier_linear.predict(summaries_test)
print_classification_report(ydf_test, labels_pred)

Training linear SVC


In [30]:
mlb = MultiLabelBinarizer()
y_enc = mlb.fit_transform(y_train)

random_forest = train_randomForest(X_train, y_train)
pickle_file('random_forest', random_forest)
labels_pred = random_forest.predict(X_test)
print_classification_report(y_test, labels_pred)

Training random forest


ValueError: Multioutput target data is not supported with label binarization

In [20]:
decision_tree = train_decisionTree(X_train, y_train)
pickle_file('decision_tree', decision_tree)
labels_pred = decision_tree.predict(X_test)
print_classification_report(y_test, labels_pred)

Training decision tree
             precision    recall  f1-score   support

          0       0.67      0.68      0.67      6542
          1       0.42      0.39      0.40      2257
          2       0.39      0.37      0.38      2000
          3       0.49      0.44      0.46       835
          4       0.39      0.36      0.37      1657
          5       0.30      0.27      0.28       726
          6       0.29      0.23      0.26       968
          7       0.24      0.19      0.21       869
          8       0.23      0.19      0.21       725
          9       0.39      0.38      0.39      2176
         10       0.56      0.47      0.51       821
         11       0.46      0.41      0.43       215
         12       0.43      0.41      0.42      2244
         13       0.45      0.39      0.42       520
         14       0.53      0.49      0.51       343
         15       0.34      0.31      0.33      1047

avg / total       0.47      0.45      0.46     23945



In [26]:
adaboost_decision_tree = train_adaboost_decision_tree(X_train, y_train)
labels_pred = adaboost_decision_tree.predict(X_test)
pickle_file('adaboost_decision_tree', adaboost_decision_tree)
print_classification_report(y_test, labels_pred)

Training adaboost decision tree


KeyboardInterrupt: 